In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 
This notebook was run at commit:
``

# Input-Output Test
Previous input-output tests were done with the old DSelector event selection, so we need to repeat the I/O study with our new FSRoot-based event selection. We'll also reduce the binning to 10 MeV to match that of data, but we'll need to integrate over the entire $0.1\leq -t \leq 1.0$ range due to the lack of statistics for this signal MC dataset. This is safe to do, as there is no t-dependent physics generated in the dataset aside from an exponential curve, so binning finer in $-t$ only changes the statistics. The signal MC was generated with the following waveset:
* $b_1(1235)$ and $\rho(1450)$ Breit-Wigners fixed to their PDG values
  * an isotropic background is included, but is so small it's negligible
* No `OmegaDalitz` amplitudes for changing the dalitz distribution of the $\omega$ and its corresponding $\lambda$ distribution
* No $D/S$ ratio that would be typically associated with the $b_1$
* Only in the PARA 0 orientation

We mimic the waveset here, but will be doing a mass-independent fit, so no Breit-Wigner terms are used. The goal of this notebook is to see if any of our fits, after having passed through detector simulation and event selection, significantly fail to find the true values we generated. See the associated analysis note for details on how we extract the truth information from the generated data to compare to for our mass-independent fits. 

Most bins were performed with 100 randomized fits and 100 bootstrap fits. Some bins above 1.5 GeV required more than 100 randomized fits to have a successfully converged one. Two bins currently unfortunately fail to converge even with 5000 randomized fit attempts, though they are significantly above the generated amplitudes such that difficulties measuring them is to be expected. Fits are performed in 10 MeV mass-independent bins.

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as stats

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/io-tests/thin-bins/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
%%bash
# print out truth YAML file used to submit truth fits
cat $STUDY_DIR/truth_submission.YAML

In [ ]:
# load in preprocessed results
with open(f"{STUDY_DIR}/t_0.1-1.0/preprocessed_results.pkl", "rb") as f:
    data = pkl.load(f)
    results = ResultManager(**data)

In [ ]:
results.summary()

## Analysis

### Standard Plots
Lets view the standard set of plots to view how our model performed overall

In [ ]:
results.plot.intensity.jp()

In [ ]:
results.plot.intensity.waves()

In [ ]:
results.plot.intensity.waves(fractional=True)

In [ ]:
results.plot.diagnostic.matrix()

In [ ]:
results.plot.intensity.moments()

In [ ]:
# lets also make a quick plot of just the truth lines of the JP plot, for reference
# this is just a rip from plot.intensity.jp(), so the methods may look odd here
colors = plt.colormaps["Dark2"].colors  # type: ignore
jp_map = {
    "Bkgd": {"color": colors[0], "marker": "."},
    "1p": {"color": colors[2], "marker": "o"},
    "1m": {"color": colors[3], "marker": "s"},
}
for d in jp_map.values():
    d.update({"markersize": 6})

fig, ax = plt.subplots()

# acceptance correct the data points using the efficiency, if needed
if results.is_acceptance_corrected:
    efficiency = (
        results.fit_df["detected_events"] / results.fit_df["generated_events"]
    )
else:
    efficiency = 1.0

mass = results.plot.intensity._masses
bin_width = results.plot.intensity._bin_width

# plot data
ax.errorbar(
    x=mass,
    xerr=bin_width / 2,
    y=results.data_df["events"].div(efficiency),
    yerr=results.data_df["events_err"].div(efficiency),
    marker=".",
    linestyle="",
    color="black",
    label="Total (GlueX Phase-I)",
)

# plot jp contributions for truth df as continuous lines
if results.truth_df is not None:
    for jp, props in jp_map.items():
        l = (
            utils.convert_amp_name(jp)
            if "Bkgd" not in jp
            else "Iso. Background"
        )
        if jp in results.truth_df.columns:
            ax.plot(
                mass,
                results.truth_df[jp],
                linestyle="-",
                marker="",
                label=l,
                color=props["color"],
            )

ax.set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
ax.set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
ax.set_ylim(bottom=0.0)
ax.legend()
plt.tight_layout()
plt.minorticks_on()
plt.savefig(f"{STUDY_DIR}/t_0.1-1.0/jp_truth_lines.pdf")

Lets run a couple easy statistical tests for all fit indices

In [ ]:
sig_amplitudes = results.get_significant_amplitudes()
significant_phases = results.get_significant_phases()
columns = list(sig_amplitudes) + list(significant_phases)

if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")

stats.normality_test(results.fit_df, results.bootstrap_df, columns, alpha=0.01)

In [ ]:
if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")
stats.bias_test(results.fit_df, results.bootstrap_df, columns)

### Broad Exploration of Problematic Region
It's clear we're having issues in the 1.0-1.3 GeV mass region. Let's explore that on a wider scale to hopefully hone in on what could be the problem. First we'll look for any consistently strong correlations in this region

In [ ]:
if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")

fit_indices = list(np.arange(0, 31, dtype=int))

all_phases = results.phase_differences
significant_phases = results.get_significant_phases(fit_indices=fit_indices)
drop_phases = all_phases - significant_phases
columns_to_drop = list(drop_phases)
columns_to_drop.extend(["m", "p", "1p", "1m", "m1p", "p1p"])

stats.report_correlations(
    results.bootstrap_df, 
    fit_indices=fit_indices, 
    report_average=True, 
    drop_columns=columns_to_drop
)
# no print out means no strong correlations are found on average

Since the D waves are so poorly defined in this region, let's see if there's any strong correlations on average, or in individual bins, for these waves

In [ ]:
cols = [wave for wave in results.coherent_sums["eJPmL"] if wave.endswith("D") or wave.endswith("S")]
fit_indices = list(np.arange(0, 31, dtype=int))

results.plot.bootstrap.correlation_matrix(columns=cols, report_average = True, fit_indices=fit_indices, pdf_path="./averaged.pdf")
results.plot.bootstrap.correlation_matrix(columns=cols, report_average = False, fit_indices=fit_indices, pdf_path="./individual.pdf")

From our earlier matrix plot, there does seem to be some positive reflectivity phases in the S and D waves which were generated flat, but resolved to having some phase motion in this region, indicating potential interference effects. Lets highlight those, but ignore the negative reflectivity since they are so poorly resolved

In [ ]:
s_waves = ["p1pmS", "p1p0S", "p1ppS"]
d_waves = ["p1pmD", "p1p0D", "p1ppD"]

# Create grid
n_rows, n_cols = len(d_waves), len(s_waves)
fig_width = n_cols * 4
fig_height = n_rows * 3.5

fig, axs = plt.subplots(
    n_rows,
    n_cols,
    sharex=True,
    figsize=(fig_width, fig_height),
    squeeze=False,
    layout="constrained",
    sharey=True,    
)

# Plot each phase difference
for row, d_wave in enumerate(d_waves):
    for col, s_wave in enumerate(s_waves):     
        ax = axs[row, col]

        # Plot the phase difference
        results.plot.phase.phase(d_wave, s_wave, ax=ax, extend_range=False, phase_kwargs={"label":""})
        
        # Set subplot titles and labels
        if row == 0:
            ax.set_title(f"{utils.convert_amp_name(s_wave)}", fontsize=12, loc="center")
        if col == 0:
            ax.set_ylabel(f"{utils.convert_amp_name(d_wave)}", fontsize=12, loc="center")
        else:
            ax.set_ylabel("")
        
        ax.set_xlabel("")

fig.supylabel(r"Phase Difference ($^{\circ}$)", fontsize=14)
fig.supxlabel(r"$\omega\pi^0$ inv. mass $(GeV)$", fontsize=14)
fig.suptitle(f"S vs D Wave Phase Differences (Positive Reflectivity)", fontsize=18)

plt.minorticks_on()
plt.show()